# **LSTM**

# **Importing necessary modules**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import text_to_word_sequence
from transformers import BertTokenizer
#from keras.applications.preprocessing import text_to_sequence  # for pre-trained embeddings
from transformers import BertTokenizer, TFBertModel  # Example using pre-trained BERT model

In [2]:
import numpy as np

In [3]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=95bf3ec57deaa1603bed6aef9d43e1ee1a3b57438e4a13f387f9f736455e4fde
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [4]:
from langdetect import detect

# **Loading Data**

In [5]:

data = pd.read_json("/content/filtered_train.json")

**Data Sample Exploration**

In [6]:
data.sample(5)

,pairs
188,"{'id': 1540, 'image': 'https://sammlung.staede..."
8,"{'id': 28, 'image': 'https://sammlung.staedelm..."
74,"{'id': 585, 'image': 'https://sammlung.staedel..."
38,"{'id': 347, 'image': 'https://sammlung.staedel..."
111,"{'id': 1013, 'image': 'https://sammlung.staede..."


In [7]:
texts = [entry['text'] for entry in data['pairs']]

In [8]:
texts[0:5]

["Dotty was wakened next morning by a variety of sounds . The mocking - bird , the canary , the hens , and Horace ' s guinea pig were astir , and wished their little world to be aware of it .",
 'These glacial streams contain no animal life , at all events no fish , till they have received the waters of warmer tributaries .',
 "Finally , rising listlessly from the couch on which he lay , his countenance irradiated with a very peculiar smile , he sauntered slowly to the door of the room . I sneered he , while making his characteristic exit , Blanche or Loo must stitch me an embroidered bib and tucker ere I pay my court to the little De Bys ! And yet , faith , jesting aside , who knows what might be effected ? Still , it 's ignoble game , a girl fresh from a foreign nursery , ' smelling of bread and butter , ' as Byron says .",
 'The people people generally appear to be very poor . Their mode of life mean ; their food coarse and indifferent , indifferent , except fish , which is excellen

In [9]:
labels = [entry['subtask1_label'] for entry in data['pairs']]

In [10]:
labels[0:5]

['NO', 'NO', 'YES', 'NO', 'NO']

In [11]:
ids = [entry['id'] for entry in data['pairs']]

**Data Pre-Processing**

In [12]:
# Define constants
MAX_LEN = 100  # Maximum sequence length
EMBEDDING_DIM = 128  # Embedding dimension
VOCAB_SIZE = 100000  # Vocabulary size

In [13]:
import unicodedata
import re

In [14]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
# Define stopwords and tokenizer
stop_words = set(stopwords.words('english'))
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [16]:
# Text preprocessing function
def preprocess_text(text):
    # Tokenize text using BERT tokenizer
    tokens = tokenizer.tokenize(text)
    # Remove stopwords and convert to lowercase
    filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words]
    return ' '.join(filtered_tokens)

In [17]:
# Preprocess text data
processed_texts = [preprocess_text(text) for text in texts]

In [18]:
# Define constants
MAX_LEN = 100  # Maximum sequence length

In [19]:
EMBEDDING_DIM = 128  # Embedding dimension

In [20]:
VOCAB_SIZE = 100000  # Vocabulary size

In [21]:
# Tokenize processed text data using BERT tokenizer
encoded_texts = [tokenizer.encode(text, add_special_tokens=True, max_length=MAX_LEN, truncation=True) for text in processed_texts]

In [22]:
# Pad sequences
padded_sequences = pad_sequences(encoded_texts, maxlen=MAX_LEN, padding='post')

In [23]:
 #Split data into training and testing sets
X_train, X_test, y_train, y_test, id_train, id_test = train_test_split(padded_sequences, labels, ids, test_size=0.2, random_state=42)

In [24]:
# Define label mapping dictionary
label_mapping = {'YES': 1, 'NO': 0}  # Adjust as per your label mapping

In [25]:
# Convert output labels to numeric format
training_labels_numeric = [label_mapping[label] for label in y_train]
testing_labels_numeric = [label_mapping[label] for label in y_test]

In [26]:
# Convert numeric labels to NumPy arrays
training_labels_numeric_np = np.array(training_labels_numeric)
testing_labels_numeric_np = np.array(testing_labels_numeric)

**Model Definition**

In [27]:
# Define LSTM-based model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LEN))
model.add(LSTM(units=64, return_sequences=True))  # LSTM layer with 64 units
model.add(Dropout(0.2))
model.add(LSTM(units=32))  # LSTM layer with 32 units
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

**Model** **Compilation**

In [28]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

**Model Training**

In [29]:
# Train the model
model.fit(X_train, training_labels_numeric_np, epochs=50, batch_size=32, validation_data=(X_test, testing_labels_numeric_np))

Epoch 1/50
7/7 [==============================] - 10s 1s/step - loss: 0.6599 - accuracy: 0.7619 - val_loss: 0.6095 - val_accuracy: 0.7358
Epoch 2/50
7/7 [==============================] - 5s 739ms/step - loss: 0.5768 - accuracy: 0.7571 - val_loss: 0.5999 - val_accuracy: 0.7358
Epoch 3/50
7/7 [==============================] - 7s 1s/step - loss: 0.5509 - accuracy: 0.7571 - val_loss: 0.5814 - val_accuracy: 0.7358
Epoch 4/50
7/7 [==============================] - 5s 773ms/step - loss: 0.5287 - accuracy: 0.7571 - val_loss: 0.5853 - val_accuracy: 0.7358
Epoch 5/50
7/7 [==============================] - 7s 1s/step - loss: 0.5023 - accuracy: 0.7571 - val_loss: 0.6134 - val_accuracy: 0.7358
Epoch 6/50
7/7 [==============================] - 5s 659ms/step - loss: 0.4576 - accuracy: 0.7571 - val_loss: 0.5938 - val_accuracy: 0.7358
Epoch 7/50
7/7 [==============================] - 4s 614ms/step - loss: 0.3634 - accuracy: 0.8048 - val_loss: 0.9339 - val_accuracy: 0.7547
Epoch 8/50
7/7 [============

**Model Evaluation**:

In [30]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, testing_labels_numeric_np)

# Print the accuracy
print("Test Accuracy:", accuracy)

2/2 [==============================] - 0s 28ms/step - loss: 2.0228 - accuracy: 0.6226
Test Accuracy: 0.6226415038108826


# RNN

# **Importing necessary modules**

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, Dropout
from transformers import BertTokenizer
import numpy as np
from langdetect import detect

**Data importing**

In [32]:
data = pd.read_json("/content/filtered_train.json")


In [33]:
texts = [entry['text'] for entry in data['pairs']]
labels = [entry['subtask1_label'] for entry in data['pairs']]
ids = [entry['id'] for entry in data['pairs']]

In [34]:
MAX_LEN = 100
VOCAB_SIZE = 100000

**Pre Processing**

In [35]:
# Tokenization and padding
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
processed_texts = [tokenizer.tokenize(text.lower()) for text in texts]
encoded_texts = [tokenizer.encode(text, add_special_tokens=True, max_length=MAX_LEN, truncation=True) for text in processed_texts]
padded_sequences = pad_sequences(encoded_texts, maxlen=MAX_LEN, padding='post')

In [36]:
X_train, X_test, y_train, y_test, id_train, id_test = train_test_split(padded_sequences, labels, ids, test_size=0.2, random_state=42)


In [37]:
# Label mapping
label_mapping = {'YES': 1, 'NO': 0}
training_labels_numeric = [label_mapping[label] for label in y_train]
testing_labels_numeric = [label_mapping[label] for label in y_test]

In [38]:
# Convert numeric labels to NumPy arrays
training_labels_numeric_np = np.array(training_labels_numeric)
testing_labels_numeric_np = np.array(testing_labels_numeric)

**Model Definition**

In [39]:
# Define RNN-based model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, 128, input_length=MAX_LEN))
model.add(SimpleRNN(units=64, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(units=32))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

**Model Compilation**

In [40]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

**Model Training**

In [41]:
model.fit(X_train, training_labels_numeric_np, epochs=50, batch_size=32, validation_data=(X_test, testing_labels_numeric_np))

Epoch 1/50
7/7 [==============================] - 5s 402ms/step - loss: 0.6561 - accuracy: 0.6333 - val_loss: 0.5967 - val_accuracy: 0.7358
Epoch 2/50
7/7 [==============================] - 3s 477ms/step - loss: 0.4101 - accuracy: 0.8333 - val_loss: 0.5504 - val_accuracy: 0.7358
Epoch 3/50
7/7 [==============================] - 4s 563ms/step - loss: 0.2215 - accuracy: 0.9476 - val_loss: 0.6177 - val_accuracy: 0.7358
Epoch 4/50
7/7 [==============================] - 3s 446ms/step - loss: 0.1132 - accuracy: 0.9762 - val_loss: 0.5740 - val_accuracy: 0.6981
Epoch 5/50
7/7 [==============================] - 3s 360ms/step - loss: 0.0524 - accuracy: 0.9857 - val_loss: 0.7478 - val_accuracy: 0.6604
Epoch 6/50
7/7 [==============================] - 2s 355ms/step - loss: 0.0286 - accuracy: 1.0000 - val_loss: 0.6090 - val_accuracy: 0.6604
Epoch 7/50
7/7 [==============================] - 4s 529ms/step - loss: 0.0265 - accuracy: 0.9905 - val_loss: 0.6784 - val_accuracy: 0.6981
Epoch 8/50
7/7 [====

**evaluation**

In [42]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, testing_labels_numeric_np)

# Print the accuracy
print("Test Accuracy:", accuracy)

2/2 [==============================] - 0s 30ms/step - loss: 0.9352 - accuracy: 0.7170
Test Accuracy: 0.7169811129570007
